In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
np.random.seed(100)

4 Physical GPUs, 1 Logical GPU


In [2]:
def conv_block(x,filters,activation,kernel_size=(3, 3),strides=(1, 1),
               padding="same",use_bias=True,use_bn=False,
               use_dropout=False,drop_value=0.5):
    
    x = layers.Conv1D(filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias)(x)
    if use_bn:
        x = layers.BatchNormalization()(x)
    x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_discriminator_model(input_shape = (256,3)):
    
    input_ = layers.Input(shape=input_shape)
    
    x = conv_block(input_,64,kernel_size=20,strides=2,use_bn=False,use_bias=True,
        activation=layers.LeakyReLU(0.2),use_dropout=False,drop_value=0.3)
    
    x = conv_block(x,128,kernel_size=10,strides=2,use_bn=False,activation=layers.LeakyReLU(0.2),
        use_bias=True,use_dropout=True,drop_value=0.3)
    
    x = conv_block(x,128,kernel_size=10,strides=2,use_bn=False,
        activation=layers.LeakyReLU(0.2),use_bias=True,use_dropout=True,drop_value=0.3)
    
    x = conv_block(x,128,kernel_size=10,strides=2,use_bn=False,
        activation=layers.LeakyReLU(0.2),use_bias=True,use_dropout=True,drop_value=0.3)
    
    x = layers.Flatten()(x)
    
    x = layers.Dropout(0.2)(x)
    
    x = layers.Dense(1)(x)

    d_model = keras.models.Model(input_, x, name="discriminator")
    
    return d_model
def upsample_block(x,filters,activation,kernel_size=20, strides=1, up_size=2, 
                    padding="same",use_bn=False,use_bias=True,
                    use_dropout=False,drop_value=0.3):
    
    x = layers.UpSampling1D(up_size)(x)
    
    x = layers.Conv1D(filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias)(x)

    if use_bn:
        x = layers.BatchNormalization()(x)

    if activation:
        x = activation(x)
        
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    
    return x


def get_generator_model(latent_size=100,first_dense_shape=128,mean_image_shape=256):
    noise = layers.Input(shape=(latent_size,))
    mean_ts = layers.Input(shape=(mean_image_shape,3))
    x = layers.Dense(first_dense_shape, use_bias=False)(noise)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Reshape((128,1))(x)
    
    x = upsample_block(x,128, layers.LeakyReLU(0.2), strides=1, use_bias=False,
                       use_bn=True, padding="same", use_dropout=False)
    
    mean_ts1 = conv_block(mean_ts,64,kernel_size=40,strides=1,use_bn=False,use_bias=True,
                   activation=layers.LeakyReLU(0.2),use_dropout=False,drop_value=0.3)
    
    x = layers.Concatenate(axis=2)([x,mean_ts1])
    
    x = upsample_block(x, 64, layers.LeakyReLU(0.2), strides=1, use_bias=False,
                       use_bn=False, padding="same", use_dropout=False)
    
    x = conv_block(x,64,kernel_size=40,strides=2,use_bn=False,use_bias=True,
                   activation=layers.LeakyReLU(0.2),use_dropout=False,drop_value=0.3)
    
    x = conv_block(x,3,kernel_size=20,strides=1,use_bn=False,use_bias=True,
                  activation=layers.Activation('tanh'),use_dropout=False,drop_value=0.3)
    
    g_model = keras.models.Model([noise,mean_ts], x, name="generator")
    
    
    return g_model

import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

# for p in range(1):
p = 0
X,y_hr,y_participant,y_activity,y_time = pickle.load(open('../data/heart_rate_tabular_data_ppg_dalia.p','rb'))
print(X.shape)
activity_dict1 = {'No Label':-1,'Sitting':0,'Stairs':1,'Soccer':2,
                'Cycling':3,'Driving':4,'Lunch':-1,'Walking':5,
                'Working':-1}
y_activity = np.array([activity_dict1[a] for a in y_activity])
X = X[:,:,1:].reshape(-1,256,3)
print(X.shape)
act_value = 5
X = X[np.where(y_activity==act_value)[0]]
print(X.shape)
min1,max1 = np.min(X),np.max(X)
cc = np.max(np.abs(X))
X = X/cc
k = 1
print(X.shape)
arrays = np.array_split(X,k,axis=1)
print(X.shape)
X1 = []
for a in arrays:
    X1.append(np.concatenate([np.mean(a,axis=1).reshape(a.shape[0],-1,a.shape[2])]*a.shape[1],axis=1))
X1 = np.concatenate(X1,axis=1)
BATCH_SIZE = 200
batch_size = BATCH_SIZE
dataset = tf.data.Dataset.from_tensor_slices((X,X1)).batch(BATCH_SIZE)

(25144, 256, 4)
(25144, 256, 3)
(4696, 256, 3)
(4696, 256, 3)
(4696, 256, 3)


In [3]:
X1.shape,X.shape

((4696, 256, 3), (4696, 256, 3))

In [4]:
def gradient_penalty(batch_size, real_images, fake_images,discriminator):
    """ Calculates the gradient penalty.

    This loss is calculated on an interpolated image
    and added to the discriminator loss.
    """
    # Get the interpolated image
    alpha = tf.random.normal([batch_size, 1, 1], 0.0, 1.0)
    fake_images = tf.cast(fake_images,tf.float32)
    real_images = tf.cast(real_images,tf.float32)
#     print(fake_images.shape,real_images.shape)
    diff = fake_images - real_images
    interpolated = real_images + alpha * diff
#     print(interpolated)
    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        # 1. Get the discriminator output for this interpolated image.
        pred = discriminator(interpolated, training=True)

    # 2. Calculate the gradients w.r.t to this interpolated image.
    grads = gp_tape.gradient(pred, [interpolated])[0]
    # 3. Calculate the norm of the gradients.
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return -real_loss +fake_loss

def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)

In [5]:
noise_dim = 100
BATCH_SIZE = 200
epochs = 60
d_steps = 5
latent_dim = noise_dim
gp_weight = 10

g_model = get_generator_model()
g_model.summary()
d_model = get_discriminator_model(input_shape=(256,3))
d_model.summary()

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          12800       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128)          512         dense[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 128)          0           batch_normalization[0][0]        
__________________________________________________________________________________________

In [6]:
import warnings
import os
from keras.models import load_model
from sklearn.metrics import classification_report,accuracy_score
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
generator_optimizer = keras.optimizers.RMSprop(
    learning_rate=0.0002)
discriminator_optimizer = keras.optimizers.RMSprop(
    learning_rate=0.0002)

def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return -real_loss +fake_loss

# Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)

activity_estimator = load_model('../model_files/activity_estimator_8_secs_dalia.hdf5')
for k in range(epochs):
    for i,real_images in enumerate(dataset):
        mean_images = real_images[1]
        real_images = real_images[0]
        batch_size = mean_images.shape[0]
        for i in range(d_steps):
            random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
            with tf.GradientTape() as tape:
                # Generate fake images from the latent vector
                fake_images = g_model([random_latent_vectors,mean_images], training=True)
                # Get the logits for the fake images
                fake_logits = d_model(fake_images, training=True)
                # Get the logits for the real images
                real_logits = d_model(real_images, training=True)
                # Calculate the discriminator loss using the fake and real image logits
                d_cost = discriminator_loss(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
                gp = gradient_penalty(batch_size, real_images, fake_images,discriminator=d_model)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, d_model.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            discriminator_optimizer.apply_gradients(zip(d_gradient, d_model.trainable_variables))
        
        random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
        y_walking = tf.zeros((batch_size))+act_value
        activity_loss = tf.keras.losses.SparseCategoricalCrossentropy()
        with tf.GradientTape() as tape:
            generated_images = g_model([random_latent_vectors,mean_images], training=True)
            gen_img_logits = d_model(generated_images, training=True)
            generated_mean = tf.concat([tf.reshape(tf.reduce_mean(generated_images,axis=1),
                                                   (generated_images.shape[0],-1,generated_images.shape[2]))]*generated_images.shape[1],axis=1)
            
            generated_activity = activity_estimator(generated_images*cc,training=True)
            g_loss = generator_loss(gen_img_logits) + tf.reduce_mean(tf.keras.losses.mean_squared_error(mean_images,generated_mean))
            g_loss = g_loss+ activity_loss(y_walking.numpy(),generated_activity.numpy())
        gen_gradient = tape.gradient(g_loss, g_model.trainable_variables)
        generator_optimizer.apply_gradients(zip(gen_gradient, g_model.trainable_variables))
    
    g_model.save('./walking_generator_epoch_/'+str(k)+'.hdf5')
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    
    generated_images_numpy = generated_images.numpy()
    generated_images = g_model([random_latent_vectors,mean_images], training=True)
    plt.figure()
    img = generated_images_numpy[0]*cc*64
    mean_img = mean_images.numpy()[0]*cc*64
    plt.plot(img)
    plt.plot(mean_img,'--')
    plt.ylim([min1*64,max1*64])
    plt.savefig("./images/generated_img_{i}.png".format(i=k))
    plt.close('all')

In [7]:
X,y_hr,y_participant,y_activity,y_time = pickle.load(open('../data/heart_rate_tabular_data_ppg_dalia.p','rb'))
activity_dict1 = {'No Label':-1,'Sitting':0,'Stairs':1,'Soccer':2,
                'Cycling':3,'Driving':4,'Lunch':-1,'Walking':5,
                'Working':-1}
y_activity = np.array([activity_dict1[a] for a in y_activity])
X = X[:,:,1:].reshape(-1,256,3)
act_value = 5
X = X[np.where(y_activity==act_value)[0]]
min1,max1 = np.min(X),np.max(X)
cc = np.max(np.abs(X))
X = X/cc
k = 1
arrays = np.array_split(X,k,axis=1)
X1 = []
for a in arrays:
    X1.append(np.concatenate([np.mean(a,axis=1).reshape(a.shape[0],-1,a.shape[2])]*a.shape[1],axis=1))
X1 = np.concatenate(X1,axis=1)
y_participant = y_participant[np.where(y_activity==act_value)[0]]
y_activity = y_activity[np.where(y_activity==act_value)[0]]

In [11]:
i=600
plt.plot(X[i]*cc*64)
plt.plot(X1[i]*cc*64,'--')
plt.ylim([min1*64,max1*64])
plt.savefig("./images/real_img_{i}.png".format(i=i))
plt.close('all')

In [12]:
path = './walking_generator_epoch_/'
for f in os.listdir(path):
    g_model = load_model(path+f)
    random_latent_vectors = tf.random.normal(shape=(X.shape[0], latent_dim))
    generated_X = g_model([random_latent_vectors,X1], training=False)


    model = load_model('../model_files/person_estimator_8_secs_dalia_final.hdf5')

    # model.summary()
    y_pred_original = model.predict(X*cc).argmax(axis=1)
    y_pred_gan = model.predict(generated_X*cc).argmax(axis=1)


    
    activity_estimator = load_model('../model_files/activity_estimator_8_secs_dalia.hdf5')

    y_activity1 = activity_estimator.predict(generated_X*cc).argmax(axis=1)
    y_activity2 = activity_estimator.predict(X*cc).argmax(axis=1)
    print(accuracy_score(y_participant,y_pred_original),'original',accuracy_score(y_participant,y_pred_gan),'GAN',
          'activity_original',len(np.where(y_activity2==act_value)[0])/len(y_activity2),
          'activity_pred',len(np.where(y_activity1==act_value)[0])/len(y_activity1),
          ) 

0.9009795570698467 original 0.06601362862010221 GAN activity_original 0.9546422487223168 activity_pred 0.09646507666098808
0.9009795570698467 original 0.22040034071550255 GAN activity_original 0.9546422487223168 activity_pred 0.2989778534923339
0.9009795570698467 original 0.06494889267461669 GAN activity_original 0.9546422487223168 activity_pred 0.17802385008517888
0.9009795570698467 original 0.12436115843270869 GAN activity_original 0.9546422487223168 activity_pred 0.2604344122657581
0.9009795570698467 original 0.06643952299829642 GAN activity_original 0.9546422487223168 activity_pred 0.09369676320272573
0.9009795570698467 original 0.20826235093696763 GAN activity_original 0.9546422487223168 activity_pred 0.3494463373083475
0.9009795570698467 original 0.06750425894378194 GAN activity_original 0.9546422487223168 activity_pred 0.0
0.9009795570698467 original 0.15502555366269166 GAN activity_original 0.9546422487223168 activity_pred 0.6220187393526405
0.9009795570698467 original 0.138202

<tf.Tensor: shape=(200,), dtype=float32, numpy=
array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.], dtype=float32)>

In [23]:
activity_estimator(generated_X*cc,training=True)

<tf.Tensor: shape=(4696, 6), dtype=float32, numpy=
array([[5.8870988e-05, 1.5368937e-01, 2.0252573e-01, 5.0645145e-03,
        1.4673933e-01, 4.9192217e-01],
       [1.6683087e-05, 6.6304496e-03, 7.3852665e-03, 7.2838092e-01,
        2.9838135e-04, 2.5728819e-01],
       [4.2903830e-06, 5.5490190e-04, 5.5177684e-04, 6.8937451e-02,
        3.5401233e-03, 9.2641139e-01],
       ...,
       [2.9513585e-06, 4.5327572e-03, 3.2724267e-05, 2.9128048e-04,
        7.3131606e-02, 9.2200863e-01],
       [7.9734752e-09, 1.2022799e-03, 4.3547225e-06, 7.3267234e-04,
        7.1318238e-04, 9.9734747e-01],
       [2.6993728e-05, 1.4591870e-05, 1.2618382e-04, 9.8730022e-01,
        4.7369234e-05, 1.2484715e-02]], dtype=float32)>

In [53]:
y_walking = tf.zeros((200))

a = tf.keras.losses.SparseCategoricalCrossentropy()
a(y_walking.numpy(),generated_activity.numpy())

<tf.Tensor: shape=(), dtype=float32, numpy=12.17736>

In [42]:
generated_activity

<tf.Tensor: shape=(200, 6), dtype=float32, numpy=
array([[3.1279203e-02, 7.3128838e-05, 6.2409025e-01, 1.2003756e-02,
        3.2905170e-01, 3.5019042e-03],
       [2.2907858e-05, 1.1204067e-02, 9.8560113e-01, 1.8217081e-05,
        2.9932682e-03, 1.6036928e-04],
       [7.3291776e-05, 3.1328050e-03, 9.9637467e-01, 1.1018338e-04,
        2.6382686e-04, 4.5078024e-05],
       ...,
       [3.0994249e-07, 9.3808484e-01, 2.0014826e-02, 1.0046176e-05,
        2.1188209e-02, 2.0701656e-02],
       [5.8765731e-10, 9.9714112e-01, 3.0014402e-04, 3.9700876e-07,
        6.4800668e-05, 2.4935910e-03],
       [1.3815179e-05, 7.3480926e-02, 8.8256365e-01, 1.2438492e-05,
        4.2613342e-02, 1.3158778e-03]], dtype=float32)>